In [1]:
import sys; sys.path.append('..')
import os
os.chdir('/Users/omersideman/dev/technion/ai-project/tuneprophet')
from tuneprophet.spotify import Spotify

os.getcwd()

spot = Spotify()


In [2]:
playlists = ['https://open.spotify.com/playlist/65LdqYCLcsV0lJoxpeQ6fW?si=f75c32235b954fbc',
            #  'https://open.spotify.com/playlist/4SWt6k4KUSNzmgRtCTzOKM?si=6e19d03bbff745b4',
             'https://open.spotify.com/playlist/37i9dQZF1DX2L0iB23Enbq?si=2dd4dae3df75498c',
             ]

In [3]:
# for playlist in playlists:
#     info = spot.get_playlist_info(playlist)
#     print(info)
    
spot.combine_to_csv(playlists, 'combined')


269
76
{'album': {'album_type': 'single', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1McMsnEElThX1knmY4oliG'}, 'href': 'https://api.spotify.com/v1/artists/1McMsnEElThX1knmY4oliG', 'id': '1McMsnEElThX1knmY4oliG', 'name': 'Olivia Rodrigo', 'type': 'artist', 'uri': 'spotify:artist:1McMsnEElThX1knmY4oliG'}], 'available_markets': ['AD', 'AE', 'AG', 'AL', 'AM', 'AO', 'AR', 'AT', 'AU', 'AZ', 'BA', 'BB', 'BD', 'BE', 'BF', 'BG', 'BH', 'BI', 'BJ', 'BN', 'BO', 'BR', 'BS', 'BT', 'BW', 'BZ', 'CA', 'CD', 'CG', 'CH', 'CI', 'CL', 'CM', 'CO', 'CR', 'CV', 'CW', 'CY', 'CZ', 'DE', 'DJ', 'DK', 'DM', 'DO', 'DZ', 'EC', 'EE', 'EG', 'ES', 'ET', 'FI', 'FJ', 'FM', 'FR', 'GA', 'GB', 'GD', 'GE', 'GH', 'GM', 'GN', 'GQ', 'GR', 'GT', 'GW', 'GY', 'HK', 'HN', 'HR', 'HT', 'HU', 'ID', 'IE', 'IL', 'IN', 'IQ', 'IS', 'IT', 'JM', 'JO', 'JP', 'KE', 'KG', 'KH', 'KI', 'KM', 'KN', 'KR', 'KW', 'KZ', 'LA', 'LB', 'LC', 'LI', 'LK', 'LR', 'LS', 'LT', 'LU', 'LV', 'LY', 'MA', 'MC', 'MD', 'ME', 'MG', 'MH'

'../data/combined.csv'

In [4]:


# spot.playlist_to_csv(tiktok_playlist, 'tiktok2023')